## Extract, Transform, Load 
This note book will be responsible for connecting to reddit api, extracting data, and storing it automatically. It will also use the python library, yfinance, to gather Yahoo Finance stock data. 

The goal is to extract data from the yfinance library, extract post content from reddit, automatically transform/clean the data and append it to a MongoDB database (via pymongo). 

Ultimately, this process has the potential to be automated.

In [315]:
# Import dependencies
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import requests
import praw
from datetime import date, timedelta
from config import KEY, CLIENT_ID, PW

In [316]:
# Create variables for API credentials
client_id = CLIENT_ID
client_k =KEY
usr_agent = 'etlAPP'
username = 'joechancey11'
pw = PW

In [317]:
# Create object for PRAW login credentials
def reddit_request():
    reddit = praw.Reddit(client_id=client_id, client_secret=client_k, user_agent=usr_agent, username=username, password=pw)
    return reddit

In [318]:
# Make reddit equal to our object
reddit = reddit_request()

In [319]:
# Choose our subreddit - Can be swapped
subreddit = reddit.subreddit("wallstreetbets")

In [320]:
# # Skip this Cell - This is a sample search so that we can get keys and understand for Reddit API is giving back results. - PRAW makes this irrelevant. 
# first_search = subreddit.search("GME", limit=5, sort='top')
# # This is commented out due to the length of the response - Feel free to uncomment to view keys. As stated above: PRAW makes this irrelevant. 
# [vars(x) for x in first_search]

In [321]:
# Create an empty DataFrame to add our data
df = pd.DataFrame(columns=['Title', 'Date', 'Upvote Ratio', 'Total Comments'])

In [322]:
# Query Reddit API for submissions that include GME
for submission in subreddit.search("GME", limit=365):
    df = df.append({'Title': submission.title, 'Date': submission.created_utc, 'Upvote Ratio': submission.upvote_ratio, 'Total Comments': submission.num_comments}, ignore_index=True)
df.head()

,Title,Date,Upvote Ratio,Total Comments
0,"Daily Popular Tickers Thread for September 16,...",1.631790e+09,0.93,12389
1,"Daily Popular Tickers Thread for September 15,...",1.631707e+09,0.92,7229
2,I just quit my job so that I could roll over m...,1.630590e+09,0.82,2079
3,Today is the day. Over 2M in my favorite stock...,1.631101e+09,0.89,1347
4,"Daily Popular Tickers Thread for September 20,...",1.632132e+09,0.92,2139


In [323]:
# Ensure our DataFrame contains GME by dropping items that do not have GME in the title
df = df[~df["Title"].str.contains("GME")==False]

In [324]:
# Convert to datetime
df['Date'] = pd.to_datetime(df['Date'], unit='s').dt.normalize()
# df.set_index('Date', drop=True, append=False, inplace=True)
df.head()

,Title,Date,Upvote Ratio,Total Comments
0,"Daily Popular Tickers Thread for September 16,...",2021-09-16,0.93,12389
1,"Daily Popular Tickers Thread for September 15,...",2021-09-15,0.92,7229
2,I just quit my job so that I could roll over m...,2021-09-02,0.82,2079
3,Today is the day. Over 2M in my favorite stock...,2021-09-08,0.89,1347
4,"Daily Popular Tickers Thread for September 20,...",2021-09-20,0.92,2139


## Yahoo Finance Data

In [325]:
# Assign GME yahoo finance data to variable
gme = yf.Ticker("GME")
# Uncomment line below if you'd like to confirm ticker data
# gme.info

In [326]:
# Get current date
current_date = date.today()
# Get the date from a year ago 
year_ago = current_date - timedelta(days=365)
# Gather stock data with the variables above - a year of stock data
hist = gme.history(start=year_ago, end=current_date)
# Drop un-needed columns that came pre-built with yfinance package
hist = hist.drop(columns=['Dividends', 'Stock Splits'])
# Make index a column
hist = hist.reset_index()
# Show DataFrame
hist.head()

,Date,Open,High,Low,Close,Volume
0,2020-09-23,10.60,10.86,9.92,10.04,10651200
1,2020-09-24,9.71,9.81,9.01,9.14,7938800
2,2020-09-25,9.19,10.18,9.10,10.02,7515200
3,2020-09-28,10.16,10.26,9.55,10.09,6764300
4,2020-09-29,10.00,10.65,9.93,10.35,5237600


## Merging Extracted Datasets Together

Below we will now merge reddit data with our stock data

In [327]:
# Merge datasets
etl_reddit_dataset = df.merge(hist, how='outer', on='Date')
# Drop NaN's from the merge - there will likely be a lot of NaN's from ticker days that went by with no posts, so this step is important
etl_reddit_dataset = etl_reddit_dataset.dropna()
# Show DataFrame
etl_reddit_dataset

,Title,Date,Upvote Ratio,Total Comments,Open,High,Low,Close,Volume
0,"Daily Popular Tickers Thread for September 16,...",2021-09-16,0.93,12389,202.330002,216.550003,201.149994,206.369995,3058200.0
1,"GME, what you missed since January",2021-09-16,0.95,302,202.330002,216.550003,201.149994,206.369995,3058200.0
2,"Daily Popular Tickers Thread for September 15,...",2021-09-15,0.92,7229,197.000000,204.869995,193.750000,204.520004,2310400.0
3,Short 2000 shares of GME here avg 203.30,2021-09-15,0.56,118,197.000000,204.869995,193.750000,204.520004,2310400.0
4,YOLO GME / AMC - I am one of you now,2021-09-15,0.89,21,197.000000,204.869995,193.750000,204.520004,2310400.0
...,...,...,...,...,...,...,...,...,...
237,Follow up: $366k worth of GME 7/16 $800 calls,2021-07-27,0.88,119,183.000000,185.000000,176.660004,178.539993,1214800.0
238,What if I told you there was a profitable comp...,2021-06-02,0.61,1143,248.880005,294.000000,244.300003,282.239990,16019200.0
239,For the GME GANG that bought at $480. Thank yo...,2021-04-30,0.92,1097,175.000000,183.800003,172.699997,173.589996,4433600.0
240,GME (what’s a memestonk?) daily Q&A panel 2/5/21,2021-02-05,0.95,34918,54.040001,95.000000,51.090000,63.770000,81345000.0


## Adding Final Dataset To Database

In [329]:
# Connection to SQL engine
connection_string = "postgres:postgres@localhost:5432/reddit_wsb_etl"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm connection
engine.table_names()

[]

In [333]:
# Push data to database
etl_reddit_dataset.to_sql(name='etl_data', con=engine, if_exists='append', index=True)

In [334]:
# Confirm data was sent (First time only)
# engine.table_names()

['etl_data']